In [1]:
! pip install imblearn

In [2]:
! pip install tensorflow

In [3]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer, roc_curve, roc_auc_score, recall_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import itertools

import tensorflow as tf
from keras.callbacks import EarlyStopping

2021-08-26 05:33:35.167799: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-26 05:33:35.167845: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
df = pd.read_csv('all_yrs_processed.csv')
df.head()

,ein,unrelbusinccd,initiationfees,grsrcptspublicuse,grsincmembers,grsincother,totcntrbgfts,totprgmrevnue,invstmntinc,txexmptbndsproceeds,...,nonpfreayr-1_8,nonpfreayr-1_9,nonpfreayr-1_11,nonpfreayr-1_12,nonpfreayr-1_13,nonpfreayr-1_14,nonpfreayr-1_15,y_term,y_liq,y_TL
0,910454080,0,0,0,0,0,935508,120315,7456,19969,...,0,0,0,0,0,0,0,0,0,0
1,111966978,0,0,0,0,0,0,5050020,153784,0,...,0,0,0,0,0,0,0,0,0,0
2,420660491,0,0,0,0,0,2945,11440,489,0,...,0,0,0,0,0,0,0,0,0,0
3,530152390,1,220425,468101,0,0,411058,2668044,29316,0,...,0,0,0,0,0,0,0,0,0,0
4,561547207,0,0,0,0,0,75,735052,783,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df = df.dropna()
print(df.shape)

(228181, 545)


In [6]:
df.loc[(df['y_term']==1) | (df['y_liq']==1),'target'] = 1

In [7]:
df.target.value_counts()

1.0    3181
Name: target, dtype: int64

In [8]:
y_cols = ['y_term', 'y_liq', 'y_TL']
X = df.drop(columns = y_cols).drop(columns = 'ein')
y = df['target']

In [9]:
# transfer to log df
numeric_df = X.apply(lambda x: np.log10(x) if np.issubdtype(x.dtype, np.number) else x)
numeric_df = numeric_df.replace(0,1)
numeric_df[np.isfinite(numeric_df)==0]=0
numeric_df.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,unrelbusinccd,initiationfees,grsrcptspublicuse,grsincmembers,grsincother,totcntrbgfts,totprgmrevnue,invstmntinc,txexmptbndsproceeds,royaltsinc,...,nonpfreayr-1_6,nonpfreayr-1_7,nonpfreayr-1_8,nonpfreayr-1_9,nonpfreayr-1_11,nonpfreayr-1_12,nonpfreayr-1_13,nonpfreayr-1_14,nonpfreayr-1_15,target
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,6.703293,5.186911,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.000000,0.000000,0.0,0.0,6.294340,6.774271,4.483887,0.0,4.683299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.000000,0.000000,0.0,0.0,0.000000,5.529430,1.913814,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,1.0,6.051747,5.339859,0.0,0.0,0.000000,6.483899,4.277128,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,1.0,0.000000,0.000000,0.0,0.0,6.930669,6.842122,3.616476,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
scaler = StandardScaler()
inputX = scaler.fit_transform(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(inputX, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((182544, 542), (45637, 542), (182544,), (45637,))

In [12]:
X_input, X_val, y_input, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [13]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y1_over = oversample.fit_resample(X_input, y_input)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
sum(X_input.isna().sum())